In [1]:
# import os
import matplotlib.pyplot as plt
import os
import pandas as pd
from amftrack.util.sys import get_analysis_folders,get_time_plate_info_from_analysis,get_time_hypha_info_from_analysis,get_global_hypha_info_from_analysis 
import numpy as np
import imageio
import os
import cv2
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
from amftrack.pipeline.final_analysis.density_wave import get_wave_fit, S, dS, wave, dwave
import matplotlib.patches as mpatches
from random import choice
import matplotlib as mpl
from amftrack.pipeline.final_analysis.density_wave import plot_single_plate,plot_single_plate_biovolume
from amftrack.pipeline.final_analysis.hypha_speed_analysis import *
from amftrack.util.plot import gridplot, make_stat
from helper import *
from load_dataframes import load_all
from plates import plates,timestep
%store -r path_figure
%matplotlib widget
%load_ext autoreload
%autoreload 2
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import scipy
import statsmodels.api as sm
import hashlib
from matplotlib.patches import Ellipse
import logging
plt.style.use('presentation.mplstyle')
logging.getLogger().setLevel(logging.WARNING)




c:\users\coren\documents\phd\code\amftrack\amftrack\util\dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [6]:
!pip install xlrd

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 96.5/96.5 kB 2.8 MB/s eta 0:00:00


In [11]:
pd.read_excel("asymmetry.xls",skiprows = [0],sheet_name = "Table S3")

,Unnamed: 0,Unnamed: 1,Genus,Ac.,Unnamed: 4,Unnamed: 5,Ent.,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Pa.,Unnamed: 29,Am.,Ar.,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,NaN,NaN,Species,mor,scr,NaN,col,con,NaN,NaN,...,NaN,NaN,NaN,occu,NaN,lept,trap,NaN,NaN,NaN
1,Trait,Host,Node ID,1,2,3,4,5,6,7,...,23,24,25,26,27,28,29,30,31,sum.CIr
2,DW,Overall,%CI,5.41,5.02,0.03,11.25,4.08,1.44,0.03,...,0.01 s+,1.8,0.77,4.31,0.66,4.91,0.07,1.36,0.01 s*,70.94
3,NaN,NaN,mean ± SD,0.936 ± 0.07,0.943 ± 0.095,0.939 ± 0.005,0.915 ± 0.142,0.98 ± 0.121,0.945 ± 0.046,0.942 ± 0.004,...,0.912 ± 0.003,0.896 ± 0.023,0.903 ± 0.011,0.892 ± 0.072,0.896 ± 0.008,0.921 ± 0.077,0.862 ± 0.016,0.894 ± 0.042,0.895 ± 0.001,NaN
4,NaN,A. mill,%CI,2.02,0.67,0.18,0.06 s*,0.24,0.62,0.00 s*,...,7.05,4.94,0.00 s*,0.48,0.29,5.66,0.06,0.00 s*,0.01 s+,65.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,NaN,NaN,mean ± SD,1.741 ± 0.27,8.962 ± 1.129,5.161 ± 5.114,6.9 ± 1.635,5.51 ± 1.078,6.246 ± 0.984,5.692 ± 0.767,...,22.54 ± 7.76,25.18 ± 3.78,17.16 ± 11.63,9.802 ± 1.442,12.69 ± 5.32,22.35 ± 11.81,0.681 ± 0.208,12.5 ± 15.39,12.59 ± 0.13,NaN
103,NaN,B. iner,%CI,0.01 s***,0.03 s+,0.46,0.01 s**,0.00 s**,0.03,0.03,...,0.47,4.22,11.59 D+,0.07 s*,6.72,0.31,0.00 s*,2.88,0.86,7.25
104,NaN,NaN,mean ± SD,2.073 ± 0.485,8.171 ± 1.602,4.961 ± 4.318,7.077 ± 0.728,4.991 ± 0.003,6.095 ± 1.477,5.516 ± 0.802,...,18.71 ± 4.35,24.09 ± 7.69,18.05 ± 8.75,10.11 ± 1.86,13.23 ± 5.74,18.34 ± 4.04,0.454 ± 0.134,10.21 ± 12.7,11.69 ± 2.14,NaN
105,NaN,M. sati,%CI,0.00 s***,0.02 s+,0.41,0.04 s+,0.01,0.00 s*,0.03,...,2.72,4.1,26.46 D**,0.15,3.78,0.42,0.00 s+,1.75,2.05,4.58
